In [46]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Ridge
from scipy.sparse import hstack

1. Загрузите данные об описаниях вакансий и соответствующих годо-
вых зарплатах из файла salary-train.csv.

In [42]:
data_train = pd.read_csv('salary-train.csv')
print(data)

                                         FullDescription LocationNormalized  \
0      International Sales Manager London ****k  ****...             London   
1      An ideal opportunity for an individual that ha...             London   
2      Online Content and Brand Manager// Luxury Reta...  South East London   
3      A great local marketleader is seeking a perman...            Dereham   
4      Registered Nurse / RGN  Nursing Home for Young...   Sutton Coldfield   
...                                                  ...                ...   
59995  As a result of continued growth, First Class S...        Whitley Bay   
59996  PHP / MVC Web Developer  MacclesfieldCirca ***...       Macclesfield   
59997  Staff Nurse, Nursing Home, Baldock White Recru...            Baldock   
59998  This is one of the best agency side opportunit...           The City   
59999  Must have CSCS card must have asbestos awarene...             London   

      ContractTime  SalaryNormalized  
0        per

2. Проведите предобработку:

• Приведите тексты к нижнему регистру.

• Замените все, кроме букв и цифр, на пробелы — это облегчит
дальнейшее разделение текста на слова. Для такой замены в
строке text подходит следующий вызов: re .sub( ’[^a−zA−Z0−9] ’ , ’␣ ’ , text . lower ())

• Примените TfidfVectorizer для преобразования текстов в век-
торы признаков. Оставьте только те слова, которые встреча-
ютсяхотябы в 5объектах(параметр min_df уTfidfVectorizer).

• Замените пропуски в столбцах LocationNormalizedиContractTime
на специальную строку ’nan’. Код для этого был приведен вы-
ше.

• Примените DictVectorizer для получения one-hot-кодирования
признаков LocationNormalized и ContractTime.

• Объедините все полученные признаки в одну матрицу "объекты-
признаки". Обратите внимание, что матрицы для текстов и
категориальных признаков являются разреженными. Для объ-
единенияих столбцов нужно воспользоваться функцией scipy.sparse.hstack

In [26]:
coloumns = ['FullDescription', 'LocationNormalized', 'ContractTime', 'SalaryNormalized']
for i in range(len(data_train['FullDescription'])):
        data_train.loc[i, 'FullDescription'] = data_train.loc[i, 'FullDescription'].lower()
print(data_train['FullDescription'])

0        international sales manager london ****k  ****...
1        an ideal opportunity for an individual that ha...
2        online content and brand manager// luxury reta...
3        a great local marketleader is seeking a perman...
4        registered nurse / rgn  nursing home for young...
                               ...                        
59995    as a result of continued growth, first class s...
59996    php / mvc web developer  macclesfieldcirca ***...
59997    staff nurse, nursing home, baldock white recru...
59998    this is one of the best agency side opportunit...
59999    must have cscs card must have asbestos awarene...
Name: FullDescription, Length: 60000, dtype: object


In [27]:
for i in range(len(data_train['FullDescription'])):
        data_train.loc[i, 'FullDescription'] = re.sub('[^a-zA-Z0-9]', ' ', data_train.loc[i, 'FullDescription'])
print(data_train['FullDescription'])

0        international sales manager london     k      ...
1        an ideal opportunity for an individual that ha...
2        online content and brand manager   luxury reta...
3        a great local marketleader is seeking a perman...
4        registered nurse   rgn  nursing home for young...
                               ...                        
59995    as a result of continued growth  first class s...
59996    php   mvc web developer  macclesfieldcirca    ...
59997    staff nurse  nursing home  baldock white recru...
59998    this is one of the best agency side opportunit...
59999    must have cscs card must have asbestos awarene...
Name: FullDescription, Length: 60000, dtype: object


In [35]:
tfidf = TfidfVectorizer(min_df=5)
description_tfid = tfidf.fit_transform(data_train['FullDescription'])

In [43]:
data_train['LocationNormalized'] = data_train['LocationNormalized'].fillna('nan')
data_train['ContractTime'] = data_train['ContractTime'].fillna('nan')

In [44]:
print(data_train['ContractTime'])

0        permanent
1        permanent
2        permanent
3        permanent
4              nan
           ...    
59995     contract
59996    permanent
59997          nan
59998    permanent
59999          nan
Name: ContractTime, Length: 60000, dtype: object


In [47]:
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data_train[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [51]:
X = hstack([description_tfid, X_train_categ])

3. Обучите гребневую регрессию с параметром alpha=1. Целевая пе-
ременная записана в столбце SalaryNormalized.

In [52]:
y = data_train['SalaryNormalized']
model = Ridge(alpha=1)
model.fit(X, y)

Ridge(alpha=1)

4. Постройте прогнозы для двух примеров из файла salary-test-mini.csv.
Значения полученных прогнозов являются ответом на задание. Ука-
жите их через пробел.

In [53]:
data_test = pd.read_csv('salary-test-mini.csv')

for i in range(len(data_test['FullDescription'])):
        data_test.loc[i, 'FullDescription'] = re.sub('[^a-zA-Z0-9]', ' ', data_test.loc[i, 'FullDescription']).lower()

description_tfid_test = tfidf.transform(data_test['FullDescription'])

data_test['LocationNormalized'] = data_test['LocationNormalized'].fillna('nan')
data_test['ContractTime'] = data_test['ContractTime'].fillna('nan')

X_test_categ = enc.transform(data_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

X_test = hstack([description_tfid_test, X_test_categ])

predictions = model.predict(X_test)


In [56]:
print(f'{round(predictions[0], 2)} {round(predictions[1], 2)}')

56574.04 37193.74
